In [1]:
%pip install tensorflow matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import os

In [4]:
# Set up data paths
train_dir = 'data/train/'
test_dir = 'data/test/'

In [6]:
# Image data generator for real-time data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,  # Increase rotation range
    width_shift_range=0.3,  # Increase shift range
    height_shift_range=0.3,
    shear_range=0.3,  # Increase shear range
    zoom_range=0.3,  # Increase zoom range
    horizontal_flip=True,
    vertical_flip=True,  # Add vertical flip
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)


In [7]:
# Create generators for training and test datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Resize images to a common size
    batch_size=32,
    class_mode='categorical'  # Since it's a multiclass classification
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

Found 1372 images belonging to 4 classes.
Found 99 images belonging to 4 classes.


In [9]:
# Generate the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.6),  # Dropout for regularization
    layers.Dense(len(train_generator.class_indices), activation='softmax')  # Output layer for multiclass
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [10]:
%pip install scipy


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=200,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

c:\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 772ms/step - accuracy: 0.2888 - loss: 1.5753

c:\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


42/42 ━━━━━━━━━━━━━━━━━━━━ 38s 838ms/step - accuracy: 0.2899 - loss: 1.5705 - val_accuracy: 0.4688 - val_loss: 1.2309
Epoch 2/200
 1/42 ━━━━━━━━━━━━━━━━━━━━ 11s 284ms/step - accuracy: 0.4375 - loss: 1.1688

c:\Python311\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.4375 - loss: 1.1688 - val_accuracy: 0.4167 - val_loss: 1.3254
Epoch 3/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 22s 511ms/step - accuracy: 0.4112 - loss: 1.2190 - val_accuracy: 0.4271 - val_loss: 1.2398
Epoch 4/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.5312 - loss: 1.1797 - val_accuracy: 0.4583 - val_loss: 1.2669
Epoch 5/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 22s 523ms/step - accuracy: 0.4700 - loss: 1.1619 - val_accuracy: 0.5104 - val_loss: 1.1743
Epoch 6/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.5000 - loss: 1.2637 - val_accuracy: 0.5208 - val_loss: 1.1730
Epoch 7/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 21s 504ms/step - accuracy: 0.4880 - loss: 1.1400 - val_accuracy: 0.4792 - val_loss: 1.2890
Epoch 8/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5000 - loss: 1.0003 - val_accuracy: 0.5417 - val_loss: 1.1894
Epoch 9/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 21s 497ms/step - accuracy: 0.5133 - loss: 1.1042 - val_accuracy: 0.5625

In [12]:
test_loss, test_acc = model.evaluate(validation_generator)
print(f"Test accuracy: {test_acc}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 116ms/step - accuracy: 0.7688 - loss: 0.6643
Test accuracy: 0.747474730014801


In [13]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_snake(image_path):
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)  # Convert image to array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize the image

    predictions = model.predict(img_array)
    class_idx = np.argmax(predictions[0])
    class_label = list(train_generator.class_indices.keys())[class_idx]
    return class_label

# Example usage
result = predict_snake(r'data/test/Bungarus multicinctus/7ddedcd0504341d7a7820d0480631191.jpg')
print(f'The snake species is: {result}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
The snake species is: Bungarus multicinctus


In [14]:
import json

In [15]:
# Save the model
os.makedirs("models", exist_ok=True)
model.save("models/snakeimageclassifier.keras")

In [16]:
# Save class names as a JSON array
snake_names = list(train_generator.class_indices.keys())  # Extract class names as an array
with open("models/snake_class_names.json", "w") as f:
    json.dump(snake_names, f)

print("Model and class names saved successfully!")

Model and class names saved successfully!
